# **Datascience introduction - Capstone Project**

### **Week 3 - Segmenting and clustering**

#### **Install packages**

Intall the webpage scraping package 'Beautiful Soup' and the html5 parser package

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge html5lib --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.2            |           py36_0          59 KB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         202 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.2-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.0-py36_0 conda-forge


soupsieve-1.9.2     

**Import libraries**

Apart from the usual libraries like pandas and numpy, the following libraries are needed and should thus be imported: 
- webpage sraping (BeautifulSoup)
- getting and posting html requests
- creating a csv file and writing to a csv file

In [3]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

**Scrape webpage and write table contents to csv file**

The scraping process is coded as follows:
- The url of the wikipage is passed as an argument to an html get request. The request is submitted and the resulting response is stored in the variable 'source'. 
- The source variable is passed along with an html parser to the BeautifulSoup constructor. The output is a tree containing all the parsed html tags. 
- With the soup's find methods the tree can be traversed using for loops. The tags that we are looking for are the table tag or the tablebody tag with the postcode data. We are specifically looking for the table rows (tag tr) and within the rows the header fields (tag th) and detail fields (tag td). 
- The text of the postal codes and the non assigned borough and neighbourhood names can be directly retrieved. The text values of assigned borough and neighbourhood names are inside an anchor tag. 
- The retrieved texts for each table row are written to a list.
- A csv file is created and opened in write mode. Every table row list is written to the csv.
- Trailing newline characters in text values are replaced with an empty string.
- After processing all the rows the csv file is closed.
- Most of the code is surrounded by a try-catch-finally clause because unexpected behavior may occur when scraping and the file resource always has to be closed.

In [4]:
# send an html get request and save the response in variable source
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# pass the source variable and a html parser to the Beautiful Soup constructor, the html document is parsed and transformed 
# into a tree of python objects, soup is the top level object
soup = BeautifulSoup(source, 'html5lib')
# look for the table tag within class wikitable sortable, then grab the table body
table_body = soup.find('table', class_='wikitable sortable').tbody
csv_filename = 'm_postal_code_scrape.csv'

# create a csv file - the header and detail rows will be written as records 
csv_file = open(csv_filename, 'w') # the 'w' indicates that the file is opened form writing
csv_writer = csv.writer(csv_file,delimiter=';')

#put scraper code inside try except clause just in case an unexpected situation may occur
try:
    for row in table_body.find_all('tr'):
        #first check the table header fields
        headerrow_list = [] #declare list for header fields within tablerow
        for header_cell in row.find_all('th'):
            if header_cell and header_cell.text: #check for empty element and epmty text
                # add text to headerrow list and remove any trailing newline characters
                headerrow_list.append(header_cell.text.replace('\n', ''))
        if len(headerrow_list) > 0: # only write a record if there are elements in the list
            csv_writer.writerow(headerrow_list)
        detailrow_list = [] #declare list for detail fields within tablerow
        for detail_cell in row.find_all('td'):
            anchor = detail_cell.find('a')
            # extract borough or neighborhood name from anchor tag
            if anchor and anchor.text:
                # add  anchor text to detailrow list and remove any trailingnewline characters
                detailrow_list.append(anchor.text.replace('\n', ''))
            # extract postcode or 'Not assigned'
            elif detail_cell and detail_cell.text:
                # add text to detailrow list and remove any trailing newline character
                detailrow_list.append(detail_cell.text.replace('\n', ''))
        if len(detailrow_list) > 0:  # only write a record if there are elements in the list
            csv_writer.writerow(detailrow_list)
except Exception as e:
    print(e) # print the exception
finally:
    # close resources
    csv_file.close()

**Load csv file into dataframe**

In [5]:
df = pd.read_csv(csv_filename, sep = ';')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Clean up the dataset and aggregate the dataset**

Data is cleaned up and neighbourshoods sharing the same postalcode and borough are aggregated and concatenated.

In [6]:
# replace column name Postcode with PostalCode
df.rename(columns = {'Postcode':'PostalCode'},inplace=True)
# remove rows with both Borough and Neighbourhood 'Not assigned'
df = df[(df.Borough != 'Not assigned') | (df.Neighbourhood != 'Not assigned')]
# replace Neigbourhood 'Not assigned' value with Borough value
df.Neighbourhood.replace('Not assigned',df.Borough,inplace=True)
# aggregate dataframe on PostalCode and Borough, concatenate Neighbourhood values, separated by comma
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

df.head(20)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**Number of rows in dataframe**

In [11]:
print("Number of rows and columns in dataframe: ",df.shape)

Number of rows and columns in dataframe:  (103, 3)
